# Verifying the Dataset

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import PIL


## Constants

In [ ]:
UNKNOWN = 'Unknown'
VALUES = ['5', '10', '25', '50', '100']
SIDES = ['Up', 'Down']
EDITIONS = ['Old', 'New']
SORTED_COINS_DIR = './sorted_coins'

## Load JSON

In [ ]:
def load_json(filename):
	with open(filename) as f:
		return json.load(f)

In [ ]:
import os
from typing import Dict
from ViaV2Json import ImageMetadata

filename = list(filter(lambda name: name.startswith('TCC_MBA_Coins') , os.listdir('.')))[-1]
json_data = load_json(filename)

# Image metadata is where we have the regions we need
images_data: Dict[str, ImageMetadata] = json_data['_via_img_metadata']

In [ ]:
from ViaV2Json import RegionAttributes

def is_uncertain(attributes: RegionAttributes):
	"""
	Returns true if the region is uncertain
	"""
	
	return attributes.get('Uncertain', {}).get('X', False)


## Check for classification mistakes

In [ ]:
# Until image 20220827_130606, old coins with values 5, 10 or 50 are undistinguishable when down, so they must be uncertain.
from colorama import Style

print(Style.BRIGHT + f'Classification Errors')

for metadata in images_data.values():
	filename = metadata['filename']
	
	if filename >= '20220827_130606.jpg': 
		continue
	
	for regionId, region in enumerate(metadata['regions']): 
		region_attr = region['region_attributes']
		
		side, edition, value = region_attr['Side'], region_attr['Edition'], region_attr['Value']

		if side == 'Down' and edition == 'Old' and int(value) != 25 and is_uncertain(region_attr) == False:
			print(f'{filename} - {regionId + 1}')



## Dataset Balancing Analysis


Let's verify if the dataset is balanced between classes.

### Create Dataframe


In [ ]:
# Load dataframe df with all the image data
from colorama import Fore, Back, Style

df = pd.DataFrame()

# Get number of coins for each type
for image in images_data:
    for regionId, region in enumerate(images_data[image]['regions']):
        attributes = region['region_attributes']
        # add row to dataframe with image name, region id, coin value, edition and side
        try:
            df = df.append({
                # image name is just the first 19 characters of the image path
                'ImageName': image[:19],
                'ImageId': image,
                'RegionId': regionId + 1,
                'Value': attributes.get('Value', UNKNOWN),
                'Edition': attributes.get('Edition', UNKNOWN),
                'Side': attributes.get('Side', UNKNOWN),
                'Uncertain': is_uncertain(attributes),
                #'FullName': f'{attributes["Value"]} {attributes["Edition"]} {attributes["Side"]}'
            }, ignore_index=True)
        except:
            print(Fore.RED + Style.BRIGHT +
                  f'Error on image {image[:19]} region {regionId + 1}', )

df.head()


### Remove uncertain classes

In [ ]:
# Those classes are the ones that we are not sure about the value of the coin.
print(f'Number of coins: {len(df)}')
df = df[df['Uncertain'] == 0]
print(f'Number of coins without uncertain: {len(df)}')

### Analyze the distribution of coins

In [ ]:
# Create charts
figsize = (13,13)
# Create a pie chart for the coin values
plt.subplot(1, 3, 1)
plt.title('Coin Values')
df[df['Value'] != UNKNOWN]['Value'].value_counts().plot.pie(autopct='%1.1f%%', figsize=figsize)

# Create a pie chart for the coin sides
plt.subplot(1, 3, 2)
plt.title('Coin Sides')
df[df['Side'] != UNKNOWN]['Side'].value_counts().plot.pie(autopct='%1.1f%%', figsize=figsize)

# Create a pie chart for the coin editions
plt.subplot(1, 3, 3)
plt.title('Coin Editions')
df[df['Edition'] != UNKNOWN]['Edition'].value_counts().plot.pie(autopct='%1.1f%%', figsize=figsize)

## Verify if the coins are correctly classified


In [ ]:
from PIL import Image, ImageOps, ImageDraw, ImageFont
from ViaV2Json import ShapeAttributes, get_bounding_box

def crop_region(image: Image.Image, region_id: int, shape: ShapeAttributes) -> Image.Image:
    """
    Crops the region from the image, and returns the cropped image.
    """

    # Get the bounding box for the region
    topleft, bottomright = get_bounding_box(shape)

    return image.crop((*topleft, *bottomright))


In [ ]:
from itertools import product
from PIL import Image
from typing import Iterable, Tuple, List

import os
import shutil

# Remove dir if it exists
if os.path.exists(SORTED_COINS_DIR):
    # Remove recursively
    shutil.rmtree(SORTED_COINS_DIR)

os.mkdir(SORTED_COINS_DIR)

count = len(images_data.values())

for image_idx, metadata in enumerate(images_data.values()):
    filename = metadata['filename']
    percent = int((image_idx + 1) / count * 100)
    
    print(f'Processing {filename}... ({percent}%)')

    with Image.open('./Moedas/' + filename) as image:
        image = ImageOps.exif_transpose(image)

        for region_id, region in enumerate(metadata['regions']):

            attributes = region['region_attributes']
            directory = f'{SORTED_COINS_DIR}/{attributes.get("Value", UNKNOWN)}'

            if not os.path.isdir(directory):
                os.mkdir(directory)

            crop = crop_region(image, region_id, region['shape_attributes'])
            crop.save(f'{directory}/{filename}_{region_id}.png')
